In [1]:
import tokenizers
# 创建分词器
bwpt = tokenizers.BertWordPieceTokenizer()
filepath = "./dataset.txt"
#训练分词器
bwpt.train(
    files=[filepath],
    vocab_size=1000,
    min_frequency=1,
    limit_alphabet=1000
)

bwpt.save_model('./bert_tokenizer/')
#output： ['./pretrained_models/vocab.txt']

['./bert_tokenizer/vocab.txt']

In [2]:
from transformers import BertTokenizer
bert_tokenizer = BertTokenizer(vocab_file="./bert_tokenizer/vocab.txt")

/usr/local/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd

df = pd.read_csv("./train.csv")

/tmp/ipykernel_595/2670194097.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./train.csv")


In [4]:
from transformers import (
    AutoConfig,
    AutoModelForMaskedLM,
    AutoTokenizer,DataCollatorForLanguageModeling,HfArgumentParser,Trainer,TrainingArguments,set_seed,
)
# 自己修改部分配置参数
config_kwargs = {
    "cache_dir": None,
    "revision": 'main',
    "use_auth_token": None,
    #      "hidden_size": 512,
    #     "num_attention_heads": 4,
    "hidden_dropout_prob": 0.2,
    "vocab_size": 1000
}

config = AutoConfig.from_pretrained('./bert-base-uncased-model/', **config_kwargs)
print(config)

model = AutoModelForMaskedLM.from_config(
    config=config,
)
model.resize_token_embeddings(len(bert_tokenizer))
#output:Embedding(863, 768, padding_idx=1)

BertConfig {
  "_name_or_path": "./bert-base-uncased-model/",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.2,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.33.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 1000
}



You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 1000. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(1000, 768, padding_idx=0)

In [5]:
from torch import nn
model_client = model.bert.embeddings
model_server1 = model.bert.encoder
model_server2 = model.cls

In [6]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import math
import os.path
import pandas as pd
from sklearn.model_selection import train_test_split
from glob import glob
from pandas import DataFrame

import random
import numpy as np
import os


import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import copy
import time

In [7]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    print(torch.cuda.get_device_name(0))    

#===================================================================
program = "SFLV1 Bert PreTrain on Phishing"
print(f"---------{program}----------")              # this is to identify the program in the slurm outputs files

NVIDIA A100 80GB PCIe
---------SFLV1 Bert PreTrain on Phishing----------


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def prRed(skk): print("\033[91m {}\033[00m" .format(skk)) 
def prGreen(skk): print("\033[92m {}\033[00m" .format(skk))     

#===================================================================

num_users = 5
epochs = 10
frac = 1
lr = 5e-5

In [9]:
#=====================================================================================================
#                           Client-side Model definition
#=====================================================================================================
# Model at client side

class Bert_client_side(nn.Module):
    def __init__(self):
        super(Bert_client_side, self).__init__()
        self.embeddings = copy.deepcopy(model_client)

    def forward(self, input_ids):
        embedding_output = self.embeddings(input_ids)
        return embedding_output

net_glob_client = Bert_client_side()
if torch.cuda.device_count() > 1:
    print("We use",torch.cuda.device_count(), "GPUs")
    net_glob_client = nn.DataParallel(net_glob_client)

net_glob_client.to(device)
print(net_glob_client) 

Bert_client_side(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(1000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
)


In [10]:
#=====================================================================================================
#                           Server-side Model definition
#=====================================================================================================
# Model at server side

class Bert_server_side(nn.Module):
    def __init__(self):
        super(Bert_server_side, self).__init__()
        self.encoder = copy.deepcopy(model_server1)
        self.cls = copy.deepcopy(model_server2)

    def forward(self, embedding_output):
        output_encoder = self.encoder(embedding_output).last_hidden_state
        output = self.cls(output_encoder)
        return output

net_glob_server = Bert_server_side()
if torch.cuda.device_count() > 1:
    print("We use",torch.cuda.device_count(), "GPUs")
    net_glob_server = nn.DataParallel(net_glob_server)   # to use the multiple GPUs 

net_glob_server.to(device)   

Bert_server_side(
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUActivation()
        )
        (output): BertOutput(
          (dense): Linear(in_features=3072, out_features=768, bias=True)
         

In [11]:
#===================================================================================
# For Server Side Loss and Accuracy
loss_train_collect = []
acc_train_collect = []
loss_test_collect = []
acc_test_collect = []
batch_acc_train = []
batch_loss_train = []
batch_acc_test = []
batch_loss_test = []
count1 = 0
count2 = 0

In [12]:
#====================================================================================================
#                                  Server Side Program
#====================================================================================================
# Federated averaging: FedAvg
import datetime
def FedAvg(w):
    w_avg = copy.deepcopy(w[0])
    for k in w_avg.keys():
        for i in range(1, len(w)):
            w_avg[k] += w[i][k]
        w_avg[k] = torch.div(w_avg[k], len(w))
    return w_avg


def calculate_accuracy(fx, y):
    preds = fx.max(1, keepdim=True)[1]
    correct = preds.eq(y.view_as(preds)).sum()
    acc = 100.00 *correct.float()/preds.shape[0]
    return acc

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [13]:
# to print train - test together in each round-- these are made global
acc_avg_all_user_train = 0
loss_avg_all_user_train = 0
loss_train_collect_user = []
acc_train_collect_user = []
loss_test_collect_user = []
acc_test_collect_user = []

In [14]:
w_glob_server = net_glob_server.state_dict()
w_locals_server = []

In [15]:
#client idx collector
idx_collect = []
l_epoch_check = False
fed_check = False
# Initialization of net_model_server and net_server (server-side model)
net_model_server = [net_glob_server for i in range(num_users)]
net_server = copy.deepcopy(net_model_server[0]).to(device)
#optimizer_server = torch.optim.Adam(net_server.parameters(), lr = lr)

In [16]:
criterion = nn.CrossEntropyLoss()

# Server-side function associated with Training
def train_server(fx_client, labels, l_epoch_count, l_epoch, idx, len_batch):
    global net_model_server, criterion, device, batch_acc_train, batch_loss_train, l_epoch_check, fed_check
    global loss_train_collect, acc_train_collect, count1, acc_avg_all_user_train, loss_avg_all_user_train, idx_collect, w_locals_server, w_glob_server, net_server
    global loss_train_collect_user, acc_train_collect_user, lr
    train_start_time = time.time() - t0
    net_server = copy.deepcopy(net_model_server[idx]).to(device)
    net_server.train()
    optimizer_server = torch.optim.Adam(net_server.parameters(), lr = lr)


    # train and update
    optimizer_server.zero_grad()

    fx_client = fx_client.to(device)

    #---------forward prop-------------
    output = net_server(fx_client)
    loss = criterion(output.view(-1, len(bert_tokenizer)), labels.view(-1))
    #--------backward prop--------------
    loss.backward()
    dfx_client = fx_client.grad.clone().detach()
    optimizer_server.step()

    batch_loss_train.append(loss.item())

    # Update the server-side model for the current batch
    net_model_server[idx] = copy.deepcopy(net_server)

    # count1: to track the completion of the local batch associated with one client
    count1 += 1
    if count1 == len_batch:

        loss_avg_train = sum(batch_loss_train)/len(batch_loss_train)

        batch_loss_train = []
        count1 = 0

        train_time = time.time() - train_start_time
        elapsed_train = format_time(train_time)
        prRed('Client{} Train => Local Epoch: {} \tLoss: {:.20f} \tTrain cost: {:}'.format(idx, l_epoch_count, loss_avg_train, elapsed_train))

        # copy the last trained model in the batch
        w_server = net_server.state_dict()

        # If one local epoch is completed, after this a new client will come
        if l_epoch_count == l_epoch-1:

            l_epoch_check = True                # to evaluate_server function - to check local epoch has completed or not
            # We store the state of the net_glob_server()
            w_locals_server.append(copy.deepcopy(w_server))

            # we store the last accuracy in the last batch of the epoch and it is not the average of all local epochs
            # this is because we work on the last trained model and its accuracy (not earlier cases)

            loss_avg_train_all = loss_avg_train

            # accumulate accuracy and loss for each new user
            loss_train_collect_user.append(loss_avg_train_all)

            # collect the id of each new user
            if idx not in idx_collect:
                idx_collect.append(idx)
                #print(idx_collect)

        # This is for federation process--------------------  所有客户端的学习都完成一轮，重新更新统计参数
        if len(idx_collect) == num_users:
            fed_check = True                                                  # to evaluate_server function  - to check fed check has hitted
            # Federation process at Server-Side------------------------- output print and update is done in evaluate_server()
            # for nicer display

            w_glob_server = FedAvg(w_locals_server)   # server端权重更新

            # server-side global model update and distribute that model to all clients ------------------------------
            net_glob_server.load_state_dict(w_glob_server)
            net_model_server = [net_glob_server for i in range(num_users)]

            w_locals_server = []
            idx_collect = []

            loss_avg_all_user_train = sum(loss_train_collect_user)/len(loss_train_collect_user)

            loss_train_collect.append(loss_avg_all_user_train)

            loss_train_collect_user = []

    # send gradients to the client
    return dfx_client

In [17]:
# Server-side functions associated with Testing
def evaluate_server(fx_client, labels, idx, len_batch, ell):
    global net_model_server, criterion, batch_acc_test, batch_loss_test, check_fed, net_server, net_glob_server
    global loss_test_collect, acc_test_collect, count2, num_users, acc_avg_train_all, loss_avg_train_all, w_glob_server, l_epoch_check, fed_check
    global loss_test_collect_user, acc_test_collect_user, acc_avg_all_user_train, loss_avg_all_user_train

    eval_start_time = time.time() - t0
    net = copy.deepcopy(net_model_server[idx]).to(device)
    net.eval()

    with torch.no_grad():
        fx_client = fx_client.to(device)
        #---------forward prop-------------
        output = net(fx_client)
        loss = criterion(output.view(-1, len(bert_tokenizer)), labels.view(-1))
        # calculate loss

        batch_loss_test.append(loss.item())

        count2 += 1
        if count2 == len_batch:
            loss_avg_test = sum(batch_loss_test)/len(batch_loss_test)
            eval_time = time.time() - eval_start_time
            elapsed_eval = format_time(eval_time)
            batch_loss_test = []
            count2 = 0

            prGreen('Client{} Test =>                 \tLoss: {:.20f} \tTest cost: {:}'.format(idx, loss_avg_test, elapsed_eval))

            # if a local epoch is completed
            if l_epoch_check:
                l_epoch_check = False

                loss_avg_test_all = loss_avg_test   # 一个客户端的测试效果

                loss_test_collect_user.append(loss_avg_test_all)

            # if federation is happened----------
            if fed_check:
                fed_check = False
                print("------------------------------------------------")
                print("------ Federation process at Server-Side ------- ")
                print("------------------------------------------------")

                loss_avg_all_user = sum(loss_test_collect_user)/len(loss_test_collect_user)

                loss_test_collect.append(loss_avg_all_user)
                loss_test_collect_user= []

                print("====================== SERVER V1==========================")
                print(' Train: Round {:3d} | Avg Loss {:.20f}'.format(ell, loss_avg_all_user_train))
                print(' Test: Round {:3d} | Avg Loss {:.20f}'.format(ell, loss_avg_all_user))
                print("==========================================================")

    return

In [18]:
#==============================================================================================================
#                                       Clients-side Program
#==============================================================================================================
class DatasetSplit(Dataset):
    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = list(idxs)

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        domain, atten_mask, labels = self.dataset[self.idxs[item]]
        return domain, atten_mask, labels

In [19]:
# Client-side functions associated with Training and Testing
class Client(object):
    def __init__(self, net_client_model, idx, lr, device, dataset_train = None, dataset_test = None, idxs = None, idxs_test = None):
        self.idx = idx
        self.device = device
        self.lr = lr
        self.local_ep = 1
        #self.selected_clients = []
        self.ldr_train = DataLoader(DatasetSplit(dataset_train, idxs), batch_size = 64, shuffle = True)
        self.ldr_test = DataLoader(DatasetSplit(dataset_test, idxs_test), batch_size = 64, shuffle = True)


    def train(self, net):
        net.train()
        optimizer_client = torch.optim.Adam(net.parameters(), lr = self.lr)

        for iter in range(self.local_ep):
            len_batch = len(self.ldr_train)
            for batch_idx, (ids, mask, labels) in enumerate(self.ldr_train):
                ids, mask, labels = ids.to(self.device), mask.to(self.device), labels.to(device)
                optimizer_client.zero_grad()
                #---------forward prop-------------
                fx = net(ids)
                client_fx = fx.clone().detach().requires_grad_(True)

                # Sending activations to server and receiving gradients from server
                dfx = train_server(client_fx, labels, iter, self.local_ep, self.idx, len_batch)

                #--------backward prop -------------
                fx.backward(dfx)
                optimizer_client.step()


            #prRed('Client{} Train => Epoch: {}'.format(self.idx, ell))

        return net.state_dict()

    def evaluate(self, net, ell):
        net.eval()

        with torch.no_grad():
            len_batch = len(self.ldr_test)
            for batch_idx, (ids, mask, labels) in enumerate(self.ldr_test):
                ids, mask, labels = ids.to(self.device), mask.to(self.device), labels.to(device)
                #---------forward prop-------------
                fx = net(ids)

                # Sending activations to server
                evaluate_server(fx, labels, self.idx, len_batch, ell)

            #prRed('Client{} Test => Epoch: {}'.format(self.idx, ell))

        return

In [20]:
#=====================================================================================================
# dataset_iid() will create a dictionary to collect the indices of the data samples randomly for each client
# IID HAM10000 datasets will be created based on this
def dataset_iid(dataset, num_users):
    
    num_items = int(len(dataset)/num_users)
    dict_users, all_idxs = {}, [i for i in range(len(dataset))]
    for i in range(num_users):
        dict_users[i] = set(np.random.choice(all_idxs, num_items, replace = False))
        all_idxs = list(set(all_idxs) - dict_users[i])
    return dict_users    

In [21]:
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

/tmp/ipykernel_595/2868560574.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('./train.csv')


In [22]:
train_data_domain = df_train.domain.values
test_data_domain = df_test.domain.values

In [23]:
import warnings
warnings.filterwarnings('ignore')
input_ids_train = []
attention_masks_train = []
labels_train = []
for sent in train_data_domain:
    encoded_dict = bert_tokenizer.encode_plus(
        sent,                      # Sentence to encode.
        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
        max_length = 64,           # Pad & truncate all sentences.
        pad_to_max_length = True,
        return_attention_mask = True,   # Construct attn. masks.
        return_tensors = 'pt',     # Return pytorch tensors.
    )

    # Add the encoded sentence to the list.
    input_ids_train.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks_train.append(encoded_dict['attention_mask'])
    labels = encoded_dict['input_ids']
    labels = torch.where(encoded_dict['input_ids']==bert_tokenizer.mask_token_id, labels, -100)
    labels_train.append(labels)

# Convert the lists into tensors.
input_ids_train = torch.cat(input_ids_train, dim=0)
attention_masks_train = torch.cat(attention_masks_train, dim=0)
labels_train = torch.cat(labels_train, dim=0)

input_ids_test = []
attention_masks_test = []
labels_test = []

for sent in test_data_domain:
    encoded_dict = bert_tokenizer.encode_plus(
        sent,                      # Sentence to encode.
        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
        max_length = 64,           # Pad & truncate all sentences.
        pad_to_max_length = True,
        return_attention_mask = True,   # Construct attn. masks.
        return_tensors = 'pt',     # Return pytorch tensors.
    )

    # Add the encoded sentence to the list.
    input_ids_test.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks_test.append(encoded_dict['attention_mask'])
    labels = encoded_dict['input_ids']
    labels = torch.where(encoded_dict['input_ids']==bert_tokenizer.mask_token_id, labels, -100)
    labels_test.append(labels)

# Convert the lists into tensors.
input_ids_test = torch.cat(input_ids_test, dim=0)
attention_masks_test = torch.cat(attention_masks_test, dim=0)
labels_test = torch.cat(labels_test, dim=0)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [24]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

dataset_train = TensorDataset(input_ids_train, attention_masks_train, input_ids_train)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, input_ids_test)

dict_users = dataset_iid(dataset_train, num_users)
dict_users_test = dataset_iid(dataset_test, num_users)

In [25]:
#------------ Training And Testing  -----------------
net_glob_client.train()
#copy weights
w_glob_client = net_glob_client.state_dict()
# Federation takes place after certain local epochs in train() client-side
# this epoch is global epoch, also known as rounds
t0 = time.time()
for iter in range(epochs):
    m = max(int(frac * num_users), 1)
    idxs_users = np.random.choice(range(num_users), m, replace = False)
    w_locals_client = []

    for idx in idxs_users:
        local = Client(net_glob_client, idx, lr, device, dataset_train = dataset_train, dataset_test = dataset_test, idxs = dict_users[idx], idxs_test = dict_users_test[idx])
        # Training ------------------
        w_client = local.train(net = copy.deepcopy(net_glob_client).to(device))
        w_locals_client.append(copy.deepcopy(w_client))

        # Testing -------------------
        local.evaluate(net = copy.deepcopy(net_glob_client).to(device), ell= iter)


    # Ater serving all clients for its local epochs------------
    # Fed  Server: Federation process at Client-Side-----------
    print("-----------------------------------------------------------")
    print("------ FedServer: Federation process at Client-Side ------- ")
    print("-----------------------------------------------------------")
    w_glob_client = FedAvg(w_locals_client)

    # Update client-side global model
    net_glob_client.load_state_dict(w_glob_client)
elapsed = format_time(time.time() - t0)
#===================================================================================

print("Training and Evaluation completed! total time cost: {:}".format(elapsed))

 Client0 Train => Local Epoch: 0 	Loss: 0.18370094124576652472 	Train cost: 19634 days, 5:14:26
 Client0 Test =>                 	Loss: 0.00141082876607970318 	Test cost: 19634 days, 5:14:26
 Client4 Train => Local Epoch: 0 	Loss: 0.18283404332225372535 	Train cost: 19634 days, 5:14:26
 Client4 Test =>                 	Loss: 0.00000776537967761115 	Test cost: 19634 days, 5:14:26
 Client1 Train => Local Epoch: 0 	Loss: 0.18329204777504487511 	Train cost: 19634 days, 5:14:26
 Client1 Test =>                 	Loss: 0.00000254157559148638 	Test cost: 19634 days, 5:14:26
 Client2 Train => Local Epoch: 0 	Loss: 0.18325415548582982272 	Train cost: 19634 days, 5:14:26
 Client2 Test =>                 	Loss: 0.00003106730097719890 	Test cost: 19634 days, 5:14:26
 Client3 Train => Local Epoch: 0 	Loss: 0.18288099350534692267 	Train cost: 19634 days, 5:14:26
 Client3 Test =>                 	Loss: 0.00001869482072130779 	Test cost: 19634 days, 5:14:26
---------------------------------------------

In [26]:
torch.save(w_glob_server, "./FedBert/FedEmbedding.pt")
torch.save(w_glob_client, "./FedBert/FedTransformer.pt")